In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

In [2]:
model_directory = "/content/sample_data/quora_question_pair"


In [3]:
model = AutoModelForSequenceClassification.from_pretrained(model_directory)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_directory)

# Set the device (GPU or CPU)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [11]:
# Example question pairs
question_pairs = [
    ['Who wrote "To Kill a Mockingbird"?', 'Who is the author of "To Kill a Mockingbird"?'],
    ['What is the capital of Pakistan?', 'What is the name of capital city of Pakistan?'],
    ['Where is the nearest gas station?', 'What is the location of gas station near to me?']
]

# Tokenize the question pairs
tokens = tokenizer(question_pairs, truncation=True, padding=True, return_tensors='pt')

# Move tokens to the device
tokens = tokens.to(device)


In [12]:
import numpy as np
# Get model predictions
with torch.no_grad():
    logits = model(**tokens).logits

# Convert logits to probabilities and predictions
logits = logits.cpu().numpy()
predictions = np.argmax(logits, axis=-1)

# Print the predicted classes (0 for 'not_duplicate', 1 for 'duplicate')
for i, pair in enumerate(question_pairs):
    print(f"Question Pair: {pair}")
    print(f"Prediction: {'Duplicate' if predictions[i] == 1 else 'Not Duplicate'}\n")


Question Pair: ['Who wrote "To Kill a Mockingbird"?', 'Who is the author of "To Kill a Mockingbird"?']
Prediction: Duplicate

Question Pair: ['What is the capital of Pakistan?', 'What is the name of capital city of Pakistan?']
Prediction: Duplicate

Question Pair: ['Where is the nearest gas station?', 'What is the location of gas station near to me?']
Prediction: Not Duplicate

